In [402]:
import numpy as np
from numpy import random as rnd

In [403]:
# Use dispersity (between 1 and 3) to choose which system to return a radius from

def pickRadius(dispersity):
    if (dispersity == 1):
        return (1/((np.pi)**0.5))
    if (dispersity == 2):
        a = np.array([((3/(2*(np.pi)))**0.5), ((1/(2*(np.pi)))**0.5)])
        return rnd.choice(a , p = [0.5, 0.5])
    if (dispersity == 3):
        a = np.array([((15/(16*(np.pi)))**0.5), ((5/(4*(np.pi)))**0.5)])
        return rnd.choice(a , p = [0.8, 0.2])

In [469]:
#Class for creating a disc
# Cooridinates represent middle of discs
# choose 1-3 for dispersity

class config(): 
    def __init__(self, dispersity):
        self.radius = pickRadius(dispersity)
        self.x = (rnd.rand()) * 20
        self.y = (rnd.rand()) * 20
        return

In [468]:
# should iterate attempts of N

NumberOfAttempts = 200
#select dispersity scenario of 1-3

def addDiscs(N, dispersity, overlapConstant = 1.0):
    situ = np.empty(N, dtype = "object")
    for i in range(N):
        situ[i] = config(dispersity)
        #check if disc is hitting side
        insideBox = (
            ((situ[i].x - situ[i].radius > 0) and (situ[i].x + situ[i].radius < 20)) 
            and ((situ[i].y - situ[i].radius > 0) and (situ[i].y + situ[i].radius < 20))
        )
        #checking hitting the disc before, accoutning for if its first
        if (i == 0):
            notHitting = True
        else:
            notHitting = (
            ((((situ[i].x - situ[i-1].x)**2) + ((situ[i].y - situ[i-1].y)**2))**0.5) >
            (overlapConstant * (situ[i].radius + situ[i-1].radius)) 
            )
        # replaces object with 0 if it doesn't meet conditions
        if (not insideBox or not notHitting):
            situ[i] = 0
            
        # Returns completed array after end of loop
    return situ
        

In [470]:
print (addDiscs(10, 1))

AttributeError: 'int' object has no attribute 'x'

In [419]:
test = np.empty(5, dtype = "void")



print(test)

[b'' b'' b'' b'' b'']
